In [ ]:
!pip install scikit-optimize
!pip install --upgrade tables
!git clone https://github.com/MaurizioFD/RecSys_Course_AT_PoliMi
!mv RecSys_Course_AT_PoliMi/* ../content/

fatal: destination path 'RecSys_Course_AT_PoliMi' already exists and is not an empty directory.
mv: cannot stat 'RecSys_Course_AT_PoliMi/*': No such file or directory


In [ ]:
!python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/usr/bin/python3'
Compiling [1/10]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1822:0,
                 from /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from MatrixFactorization_Cython_Epoch.c:620:
/usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy API, disable it with " \
  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function ‘__pyx_pf_32MatrixFactorization_Cython_Epoch_32Matrix

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import zipfile
dataFile = zipfile.ZipFile("gdrive/MyDrive/recommender-system-2021-challenge-polimi.zip")
dataFile.extractall('data')

In [ ]:
import pandas as pd
import numpy as np
import scipy.sparse as sps

seed = 1234
np.random.seed(seed)

def getCOOs():
    
    path = '/content/data/'
    URM_path = path + 'data_train.csv'
    URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path, 
                                    sep=",",
                                    dtype={0:int, 1:int, 2:float})
    URM_all_dataframe.columns = ["UserID", "ItemID", "Interaction"]
    URM_all_dataframe.head(n=10)

    ICM_genre_path = path + 'data_ICM_genre.csv'
    ICM_genre_all_dataframe = pd.read_csv(filepath_or_buffer=ICM_genre_path, 
                                    sep=",",
                                    dtype={0:int, 1:int, 2:float})
    ICM_genre_all_dataframe.columns = ["ItemID", "GenreID", "Match"]
    ICM_genre_all_dataframe.head(n=10)

    ICM_subgenre_path =path + 'data_ICM_subgenre.csv'
    ICM_subgenre_all_dataframe = pd.read_csv(filepath_or_buffer=ICM_subgenre_path, 
                                    sep=",",
                                    dtype={0:int, 1:int, 2:float})
    ICM_subgenre_all_dataframe.columns = ["ItemID", "SubgenreID", "Match"]
    ICM_subgenre_all_dataframe.head(n=10)

    ICM_channel_path = path + 'data_ICM_channel.csv'
    ICM_channel_all_dataframe = pd.read_csv(filepath_or_buffer=ICM_channel_path, 
                                    sep=",",
                                    dtype={0:int, 1:int, 2:float})
    ICM_channel_all_dataframe.columns = ["ItemID", "ChannelID", "Match"]
    ICM_channel_all_dataframe.head(n=10)

    ICM_event_path = path + 'data_ICM_event.csv'
    ICM_event_all_dataframe = pd.read_csv(filepath_or_buffer=ICM_event_path, 
                                    sep=",",
                                    dtype={0:int, 1:int, 2:float})
    ICM_event_all_dataframe.columns = ["ItemID", "EpisodeID", "Match"]
    ICM_event_all_dataframe.head(n=10)

    URM_all = sps.coo_matrix((URM_all_dataframe["Interaction"].values, 
                              (URM_all_dataframe["UserID"].values, URM_all_dataframe["ItemID"].values)))

    # URM_all.tocsr()

    ICM_genre_all = sps.coo_matrix((ICM_genre_all_dataframe["Match"].values, 
                              (ICM_genre_all_dataframe["ItemID"].values, ICM_genre_all_dataframe["GenreID"].values)))

    # ICM_genre_all.tocsr()

    ICM_subgenre_all = sps.coo_matrix((ICM_subgenre_all_dataframe["Match"].values, 
                              (ICM_subgenre_all_dataframe["ItemID"].values, ICM_subgenre_all_dataframe["SubgenreID"].values)))

    # ICM_subgenre_all.tocsr().data

    ICM_channel_all = sps.coo_matrix((ICM_channel_all_dataframe["Match"].values, 
                              (ICM_channel_all_dataframe["ItemID"].values, ICM_channel_all_dataframe["ChannelID"].values)))

    # ICM_channel_all.tocsr()

    ICM_event_all = sps.coo_matrix((ICM_event_all_dataframe["Match"].values, 
                              (ICM_event_all_dataframe["ItemID"].values, ICM_event_all_dataframe["EpisodeID"].values)))


    return URM_all, ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_event_all

URM_all, ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_event_all = getCOOs()
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

In [ ]:
import os

output_folder_path = "gdrive/MyDrive/result_experiments/4_SLIMBPR/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 10  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

# SLIM Model

In [ ]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

In [ ]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13645 ( 0.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13643 ( 0.1%) Users that have less than 1 test interactions


In [ ]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "epochs": Categorical([400]),
    "lambda_i": Real(1e-06, 1e-02),
    "lambda_j": Real(1e-06, 1e-02),
    "learning_rate": Real(low = 1e-4, high = 1e-1, prior = 'log-uniform'),
    "topK": Integer(50, 600),
    "sgd_mode": Categorical(["sgd", "adagrad", "adam"])
}

earlystopping_keywargs = {"validation_every_n": 5,
                          "stop_on_validation": True,
                          "evaluator_object": evaluator_validation,
                          "lower_validations_allowed": 5,
                          "validation_metric": metric_to_optimize,
                          }

In [ ]:
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = SLIM_BPR_Cython

hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_validation,
                                         evaluator_test=evaluator_test)

In [ ]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = earlystopping_keywargs     # Additiona hyperparameters for the fit function
)

In [ ]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = earlystopping_keywargs     # Additiona hyperparameters for the fit function
)

In [ ]:
rfs = False

if os.path.exists(output_folder_path + recommender_class.RECOMMENDER_NAME + "_metadata.zip"):
  rfs = True

print(rfs)

True


In [ ]:
hyperparameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path, # Where to save the results
                       output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                       resume_from_saved = rfs
                      )

SearchBayesianSkopt: Resuming 'SLIM_BPR_Recommender'... Loaded 5 configurations.
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 1.1303
Function value obtained: -0.1050
Current minimum: -0.1119
Iteration No: 2 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'epochs': 400, 'lambda_i': 1e-06, 'lambda_j': 1e-06, 'learning_rate': 0.0025565618881075703, 'topK': 600, 'sgd_mode': 'adagrad'}
SLIM_BPR_Recommender: URM Detected 2 ( 0.0%) items with no interactions.
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Available RAM is 12041.00 MB (92.69%) of 12991.00 MB, required is 1304.51 MB. Using dense matrix.
Processed 13650 (100.0%) in 0.47 sec. BPR loss is 9.23E-02. Sample per second: 28822
SLIM_BPR_Recommender: Epoch 1 of 400. Elapsed time 0.45 sec
Processed 13650 (100.0%) in 0.93 sec. BPR loss is 3.10E-01. Sample per second: 14633
SLIM_BPR_Recommender: Epoch 2

In [ ]:
from Recommenders.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

search_metadata.keys()

ValueError: ignored

In [ ]:
hyp = search_metadata["hyperparameters_best"]
hyp

In [ ]:
result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df

In [ ]:
import pandas as pd
recommender = SLIM_BPR_Cython(URM_all.tocsr()) # <-----
K = 10

recommender.fit(URM_all.tocsr(), epochs=hyp["epochs"], lambda_i=hyp["lambda_i"], lambda_j=hyp["lambda_j"], 
                learning_rate=hyp["learning_rate"], topK=hyp["topK"], gamma=hyp["gamma"], sgd_mode=hyp["sgd_mode"],
                beta_1=hyp["beta_1"], beta_2=hyp["beta_2"])

user_test_path = '../data/data_target_users_test.csv'
user_test_dataframe = pd.read_csv(filepath_or_buffer=user_test_path,
								sep=",",
								dtype={0:int})

subm_set = user_test_dataframe.to_numpy().T[0]


subm_res = {"user_id":[], "item_list":[]}

for user_id in subm_set:
	subm_res["user_id"].append(user_id)
	res = recommender.recommend(user_id, cutoff=K)
	res = ' '.join(map(str, res))
	if user_id < 3:
		print(user_id)
		print(res)
	subm_res["item_list"].append(res)


# print(subm_res)

submission = pd.DataFrame.from_dict(subm_res)
# submission

from datetime import datetime

now = datetime.now() # current date and time

submission.to_csv('../subs/submission {:%Y_%m_%d %H_%M_%S}.csv'.format(now), index=False)

NameError: ignored